In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/FYP_Python_Notebooks/Data_for_School_Computer')
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
# if ram_gb < 20:
#   print('Not using a high-RAM runtime')
# else:
#   print('You are using a high-RAM runtime!')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
pip install pyarrow

In [4]:
no_ts_fresh = pd.read_parquet("EXP_1_combined_dd_ss_raw_feat_data_no_tsfresh.parquet", engine="pyarrow")
indiv_zones_ts_fresh = pd.read_parquet("EXP_2_combined_dd_ss_raw_feat_data_indiv_zones_significant_lags_pruned.parquet", engine="pyarrow")

In [5]:
cluster_sig_lags = pd.read_parquet("EXP_3_combined_dd_ss_raw_feat_data_significant_lags_pruned_cluster_centers.parquet", engine="pyarrow")
all_covariates = pd.read_parquet("EXP_4_combined_dd_ss_all_covariates.parquet", engine="pyarrow")

In [6]:
cluster_aggregation = pd.read_parquet("EXP_5_combined_dd_ss_raw_feat_data_cluster_aggregation.parquet", engine="pyarrow")
poi_aggregation = pd.read_parquet("EXP_6_combined_dd_ss_raw_feat_data_poi_aggregation.parquet", engine="pyarrow")

# No Tsfresh

In [ ]:
no_ts_fresh

,time_idx,datetime,group,target,inflow
0,0,2021-01-01 00:00:00,4,3.0,8.0
1,1,2021-01-01 01:00:00,4,1.0,13.0
2,2,2021-01-01 02:00:00,4,2.0,9.0
3,3,2021-01-01 03:00:00,4,3.0,5.0
4,4,2021-01-01 04:00:00,4,0.0,2.0
...,...,...,...,...,...
1637653,24808,2023-10-31 16:00:00,263,126.0,153.0
1637654,24809,2023-10-31 17:00:00,263,112.0,135.0
1637655,24810,2023-10-31 18:00:00,263,99.0,119.0
1637656,24811,2023-10-31 19:00:00,263,60.0,119.0


In [ ]:
print(no_ts_fresh['datetime'][:10].tolist())

[Timestamp('2021-01-01 00:00:00'), Timestamp('2021-01-01 01:00:00'), Timestamp('2021-01-01 02:00:00'), Timestamp('2021-01-01 03:00:00'), Timestamp('2021-01-01 04:00:00'), Timestamp('2021-01-01 05:00:00'), Timestamp('2021-01-01 06:00:00'), Timestamp('2021-01-01 07:00:00'), Timestamp('2021-01-01 08:00:00'), Timestamp('2021-01-01 09:00:00')]


In [ ]:
for c in no_ts_fresh:
    print(c)

time_idx
datetime
group
target
inflow


In [ ]:
unique_values_group_no_ts_fresh = list(no_ts_fresh['group'].unique())
print(len(unique_values_group_no_ts_fresh), "\n")
print(unique_values_group_no_ts_fresh)

66 

[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]


In [ ]:
zones_time_idx_no_ts_fresh = {}
for i in unique_values_group_no_ts_fresh:
    df = no_ts_fresh[no_ts_fresh['group'] == i]
    time_idx_list = df['time_idx'].tolist()
    first_idx = time_idx_list[0]
    last_idx = time_idx_list[-1]
    zones_time_idx_no_ts_fresh[i] = [first_idx, last_idx]
    print("Group ", i, " has ", df.shape[0], " number of rows in df_ts_fresh \n")
    print("First idx: ", first_idx, "; Last idx: ", last_idx, "\n")

Group  4  has  24813  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  24812 

Group  12  has  24813  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  24812 

Group  13  has  24813  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  24812 

Group  24  has  24813  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  24812 

Group  41  has  24813  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  24812 

Group  42  has  24813  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  24812 

Group  43  has  24813  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  24812 

Group  45  has  24813  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  24812 

Group  48  has  24813  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  24812 

Group  50  has  24813  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  24812 

Group  68  has  24813  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  24812 

Group  74  

In [ ]:
encoder_len = 8
pred_len = 1

total_time_idx = no_ts_fresh[no_ts_fresh.group == 4].shape[0]

first_idx = no_ts_fresh.loc[no_ts_fresh['group'] == 4, 'time_idx'].iloc[0]

tr_stop_idx = int(0.7*total_time_idx)
val_start_idx = tr_stop_idx - encoder_len
val_stop_idx = val_start_idx + int(0.2*total_time_idx) # use (tr_stop_idx - encoder_len) as encoder from train set for validatoin
tes_start_idx = val_stop_idx - encoder_len
tes_stop_idx = total_time_idx-1

print(tr_stop_idx)
print(val_start_idx)
print(val_stop_idx)
print(tes_start_idx)
print(tes_stop_idx)

17369
17361
22323
22315
24812


In [ ]:
train_time_idx = [*range(first_idx, first_idx+tr_stop_idx+1, 1)]
print(train_time_idx[0], train_time_idx[-1])
val_time_idx = [*range(first_idx+val_start_idx, first_idx+val_stop_idx+1, 1)]
print(val_time_idx[0], val_time_idx[-1])
test_time_idx = [*range(first_idx+tes_start_idx, first_idx+tes_stop_idx+1, 1)]
print(test_time_idx[0], test_time_idx[-1])

0 17369
17361 22323
22315 24812


# Indiv Zones Tsfresh

In [7]:
indiv_zones_ts_fresh

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,inflow_diff2_per1__count_above_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_3,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0",inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,0.0,-0.517121,0.028399,-0.517109,-0.362827,-0.207694,-6.0,-0.000421,0.0,0.574259
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,1.0,-1.000000,0.028399,-0.517109,-0.362827,-0.207694,-3.0,-0.000421,1.0,0.574259
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,2.0,-0.250000,-1.000000,-0.517109,-0.362827,-0.207694,0.0,-0.000421,1.0,0.574259
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,3.0,-0.146199,-0.438596,-0.146199,-0.362827,-0.207694,2.0,-0.684211,1.0,3.750693
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,3.0,-0.131773,-0.532020,-0.131773,-0.362827,-0.207694,-3.0,-0.919951,2.0,-3.126259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,14.0,-0.421845,-0.103924,-0.421845,-0.342896,0.203789,-64.0,0.351349,11.0,0.235154
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,13.0,-0.546637,0.012524,-0.546637,-0.408289,-0.018269,10.0,0.202870,12.0,0.405906
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,14.0,-0.540946,0.001364,-0.540946,-0.411744,-0.030810,14.0,0.203590,11.0,0.406788
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,14.0,-0.443736,-0.035146,-0.443736,-0.288941,0.078741,-8.0,0.206272,11.0,0.743705


In [12]:
first_idx = indiv_zones_ts_fresh.loc[indiv_zones_ts_fresh['group'] == 13, 'time_idx'].iloc[0]
print(first_idx)

23


In [8]:
for c in indiv_zones_ts_fresh:
    print(c)

time_idx
datetime
group
target
inflow_lag_0
inflow_lag_1
inflow_lag_2
inflow_lag_3
outflow_lag_0
outflow_lag_1
outflow_lag_2
outflow_lag_3
inflow
inflow_diff1_per1
inflow_diff1_per2
inflow_diff2_per1
inflow_lag_0__fft_coefficient__attr_"real"__coeff_0
inflow_lag_0__fft_coefficient__attr_"real"__coeff_1
inflow_lag_0__skewness
inflow_lag_0__kurtosis
inflow_lag_0__count_above_mean
inflow_lag_0__count_below_mean
inflow_lag_0__autocorrelation__lag_1
inflow_lag_0__autocorrelation__lag_2
inflow_lag_0__autocorrelation__lag_3
inflow_lag_0__partial_autocorrelation__lag_1
inflow_lag_0__partial_autocorrelation__lag_2
inflow_lag_0__partial_autocorrelation__lag_3
inflow_lag_1__fft_coefficient__attr_"real"__coeff_0
inflow_lag_1__fft_coefficient__attr_"real"__coeff_1
inflow_lag_1__skewness
inflow_lag_1__kurtosis
inflow_lag_1__count_above_mean
inflow_lag_1__count_below_mean
inflow_lag_1__autocorrelation__lag_1
inflow_lag_1__autocorrelation__lag_2
inflow_lag_1__autocorrelation__lag_3
inflow_lag_1__parti

In [9]:
unique_values_group_indiv_zones_tsfresh = list(indiv_zones_ts_fresh['group'].unique())
print(len(unique_values_group_indiv_zones_tsfresh), "\n")
print(unique_values_group_indiv_zones_tsfresh)

66 

[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]


In [11]:
zones_time_idx_indiv_zones_tsfresh = {}
for i in unique_values_group_indiv_zones_tsfresh:
    df = indiv_zones_ts_fresh[indiv_zones_ts_fresh['group'] == i]
    time_idx_list = df['time_idx'].tolist()
    first_idx = time_idx_list[0]
    last_idx = time_idx_list[-1]
    zones_time_idx_indiv_zones_tsfresh[i] = [first_idx, last_idx]
    print("Group ", i, " has ", df.shape[0], " number of rows in df_ts_fresh \n")
    print("First idx: ", first_idx, "; Last idx: ", last_idx, "\n")

Group  4  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  12  has  24789  number of rows in df_ts_fresh 

First idx:  24 ; Last idx:  24812 

Group  13  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  24  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  41  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  42  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  43  has  24788  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  24812 

Group  45  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  48  has  24788  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  24812 

Group  50  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  68  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24

In [ ]:
encoder_len = 8
pred_len = 1

total_time_idx = indiv_zones_ts_fresh[indiv_zones_ts_fresh.group == 13].shape[0]

first_idx = indiv_zones_ts_fresh.loc[indiv_zones_ts_fresh['group'] == 13, 'time_idx'].iloc[0]

tr_stop_idx = int(0.7*total_time_idx)
val_start_idx = tr_stop_idx - encoder_len
val_stop_idx = val_start_idx + int(0.2*total_time_idx) # use (tr_stop_idx - encoder_len) as encoder from train set for validatoin
tes_start_idx = val_stop_idx - encoder_len
tes_stop_idx = total_time_idx-1

print(tr_stop_idx)
print(val_start_idx)
print(val_stop_idx)
print(tes_start_idx)
print(tes_stop_idx)

17353
17345
22303
22295
24789


In [ ]:
train_time_idx = [*range(first_idx, first_idx+tr_stop_idx+1, 1)]
print(train_time_idx[0], train_time_idx[-1])
val_time_idx = [*range(first_idx+val_start_idx, first_idx+val_stop_idx+1, 1)]
print(val_time_idx[0], val_time_idx[-1])
test_time_idx = [*range(first_idx+tes_start_idx, first_idx+tes_stop_idx+1, 1)]
print(test_time_idx[0], test_time_idx[-1])

23 17376
17368 22326
22318 24812


# Cluster Sig Lags for Individual Zones

In [ ]:
cluster_sig_lags

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__count_above_mean,inflow_diff2_per1__partial_autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_3,inflow_diff2_per1__partial_autocorrelation__lag_1,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0",inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis
0,98,2021-01-05 02:00:00,4,1.0,2.0,16.0,4.0,8.0,0.0,2.0,...,0.028431,-0.517167,0.0,-0.362851,-0.207721,-0.517158,4.0,-0.000421,0.0,0.574349
1,99,2021-01-05 03:00:00,4,0.0,4.0,10.0,4.0,13.0,1.0,2.0,...,0.028431,-1.000000,1.0,-0.362851,-0.207721,-0.517158,-2.0,-0.000421,1.0,0.574349
2,100,2021-01-05 04:00:00,4,0.0,0.0,5.0,5.0,9.0,0.0,0.0,...,0.486486,-0.993243,2.0,-0.362851,-0.207721,-0.517158,3.0,-0.000421,1.0,0.574349
3,101,2021-01-05 05:00:00,4,0.0,1.0,9.0,5.0,5.0,0.0,2.0,...,0.505017,-0.959866,2.0,-0.362851,-0.207721,-0.959866,3.0,-0.130435,2.0,-0.001745
4,102,2021-01-05 06:00:00,4,0.0,2.0,4.0,3.0,2.0,0.0,0.0,...,0.439394,-0.899733,3.0,-0.362851,-0.207721,-0.899733,4.0,-0.131016,2.0,1.131574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631613,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,-0.103924,-0.421845,14.0,-0.342896,0.203789,-0.421845,-64.0,0.351349,11.0,0.235154
1631614,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,0.012524,-0.546637,13.0,-0.408289,-0.018269,-0.546637,10.0,0.202870,12.0,0.405906
1631615,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,0.001364,-0.540946,14.0,-0.411744,-0.030810,-0.540946,14.0,0.203590,11.0,0.406788
1631616,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,-0.035146,-0.443736,14.0,-0.288941,0.078741,-0.443736,-8.0,0.206272,11.0,0.743705


In [ ]:
for c in cluster_sig_lags:
    print(c)

time_idx
datetime
group
target
inflow_lag_0
inflow_lag_1
inflow_lag_2
inflow_lag_3
outflow_lag_0
outflow_lag_1
outflow_lag_2
outflow_lag_3
inflow
inflow_diff1_per1
inflow_diff1_per2
inflow_diff2_per1
inflow_lag_0__fft_coefficient__attr_"real"__coeff_0
inflow_lag_0__fft_coefficient__attr_"real"__coeff_1
inflow_lag_0__skewness
inflow_lag_0__kurtosis
inflow_lag_0__count_above_mean
inflow_lag_0__count_below_mean
inflow_lag_0__autocorrelation__lag_1
inflow_lag_0__autocorrelation__lag_2
inflow_lag_0__autocorrelation__lag_3
inflow_lag_0__partial_autocorrelation__lag_1
inflow_lag_0__partial_autocorrelation__lag_2
inflow_lag_0__partial_autocorrelation__lag_3
inflow_lag_1__fft_coefficient__attr_"real"__coeff_0
inflow_lag_1__fft_coefficient__attr_"real"__coeff_1
inflow_lag_1__skewness
inflow_lag_1__kurtosis
inflow_lag_1__count_above_mean
inflow_lag_1__count_below_mean
inflow_lag_1__autocorrelation__lag_1
inflow_lag_1__autocorrelation__lag_2
inflow_lag_1__autocorrelation__lag_3
inflow_lag_1__parti

In [ ]:
unique_values_group_cluster_indiv = list(cluster_sig_lags['group'].unique())
print(len(unique_values_group_cluster_indiv), "\n")
print(unique_values_group_cluster_indiv)

66 

[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]


In [ ]:
zones_time_idx_cluster_indiv = {}
for i in unique_values_group_cluster_indiv:
    df = cluster_sig_lags[cluster_sig_lags['group'] == i]
    time_idx_list = df['time_idx'].tolist()
    first_idx = time_idx_list[0]
    last_idx = time_idx_list[-1]
    zones_time_idx_cluster_indiv[i] = [first_idx, last_idx]
    print("Group ", i, " has ", df.shape[0], " number of rows in df_ts_fresh \n")
    print("First idx: ", first_idx, "; Last idx: ", last_idx, "\n")

Group  4  has  24715  number of rows in df_ts_fresh 

First idx:  98 ; Last idx:  24812 

Group  12  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  13  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  24  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  41  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  42  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  43  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  45  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  48  has  24788  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  24812 

Group  50  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  68  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  248

In [ ]:
encoder_len = 8
pred_len = 1

total_time_idx = cluster_sig_lags[cluster_sig_lags.group == 261].shape[0]

first_idx = cluster_sig_lags.loc[cluster_sig_lags['group'] == 261, 'time_idx'].iloc[0]

tr_stop_idx = int(0.7*total_time_idx)
val_start_idx = tr_stop_idx - encoder_len
val_stop_idx = val_start_idx + int(0.2*total_time_idx) # use (tr_stop_idx - encoder_len) as encoder from train set for validatoin
tes_start_idx = val_stop_idx - encoder_len
tes_stop_idx = total_time_idx-1

print(tr_stop_idx)
print(val_start_idx)
print(val_stop_idx)
print(tes_start_idx)
print(tes_stop_idx)

17353
17345
22303
22295
24789


In [ ]:
train_time_idx = [*range(first_idx, first_idx+tr_stop_idx+1, 1)]
print(train_time_idx[0], train_time_idx[-1])
val_time_idx = [*range(first_idx+val_start_idx, first_idx+val_stop_idx+1, 1)]
print(val_time_idx[0], val_time_idx[-1])
test_time_idx = [*range(first_idx+tes_start_idx, first_idx+tes_stop_idx+1, 1)]
print(test_time_idx[0], test_time_idx[-1])

23 17376
17368 22326
22318 24812


# Covariates

In [ ]:
all_covariates

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,wind_speed,rain_1h,temp,clouds_all,snow_3h,rain_3h,humidity,city_name,weather_description,snow_1h
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,7.70,0.0,276.41,0,0.0,0.0,53,Alphabet City,sky is clear,0.0
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,8.23,0.0,275.35,0,0.0,0.0,56,Alphabet City,sky is clear,0.0
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,6.70,0.0,274.53,0,0.0,0.0,61,Alphabet City,sky is clear,0.0
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,6.20,0.0,274.13,0,0.0,0.0,63,Alphabet City,sky is clear,0.0
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,4.63,0.0,273.48,0,0.0,0.0,64,Alphabet City,sky is clear,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,4.63,0.0,282.29,20,0.0,0.0,50,Yorkville West,few clouds,0.0
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,4.12,0.0,283.77,20,0.0,0.0,47,Yorkville West,few clouds,0.0
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,3.60,0.0,283.52,75,0.0,0.0,50,Yorkville West,broken clouds,0.0
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,4.12,0.0,283.69,100,0.0,0.0,48,Yorkville West,overcast clouds,0.0


In [ ]:
for c in all_covariates:
    print(c)

time_idx
datetime
group
target
inflow_lag_0
inflow_lag_1
inflow_lag_2
inflow_lag_3
outflow_lag_0
outflow_lag_1
outflow_lag_2
outflow_lag_3
inflow
inflow_diff1_per1
inflow_diff1_per2
inflow_diff2_per1
inflow_lag_0__fft_coefficient__attr_"real"__coeff_0
inflow_lag_0__fft_coefficient__attr_"real"__coeff_1
inflow_lag_0__skewness
inflow_lag_0__kurtosis
inflow_lag_0__count_above_mean
inflow_lag_0__count_below_mean
inflow_lag_0__autocorrelation__lag_1
inflow_lag_0__autocorrelation__lag_2
inflow_lag_0__autocorrelation__lag_3
inflow_lag_0__partial_autocorrelation__lag_1
inflow_lag_0__partial_autocorrelation__lag_2
inflow_lag_0__partial_autocorrelation__lag_3
inflow_lag_1__fft_coefficient__attr_"real"__coeff_0
inflow_lag_1__fft_coefficient__attr_"real"__coeff_1
inflow_lag_1__skewness
inflow_lag_1__kurtosis
inflow_lag_1__count_above_mean
inflow_lag_1__count_below_mean
inflow_lag_1__autocorrelation__lag_1
inflow_lag_1__autocorrelation__lag_2
inflow_lag_1__autocorrelation__lag_3
inflow_lag_1__parti

In [ ]:
print(all_covariates['weather_main'][:10].tolist())

['Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear']


In [ ]:
unique_values_group_covariates = list(all_covariates['group'].unique())
print(len(unique_values_group_covariates), "\n")
print(unique_values_group_covariates)

66 

[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]


In [ ]:
zones_time_idx_covariates = {}
for i in unique_values_group_covariates:
    df = all_covariates[all_covariates['group'] == i]
    time_idx_list = df['time_idx'].tolist()
    first_idx = time_idx_list[0]
    last_idx = time_idx_list[-1]
    zones_time_idx_covariates[i] = [first_idx, last_idx]
    print("Group ", i, " has ", df.shape[0], " number of rows in df_ts_fresh \n")
    print("First idx: ", first_idx, "; Last idx: ", last_idx, "\n")

Group  4  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  12  has  24789  number of rows in df_ts_fresh 

First idx:  24 ; Last idx:  24812 

Group  13  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  24  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  41  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  42  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  43  has  24788  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  24812 

Group  45  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  48  has  24788  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  24812 

Group  50  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  68  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24

In [ ]:
encoder_len = 8
pred_len = 1

total_time_idx = all_covariates[all_covariates.group == 261].shape[0]

first_idx = all_covariates.loc[all_covariates['group'] == 261, 'time_idx'].iloc[0]

tr_stop_idx = int(0.7*total_time_idx)
val_start_idx = tr_stop_idx - encoder_len
val_stop_idx = val_start_idx + int(0.2*total_time_idx) # use (tr_stop_idx - encoder_len) as encoder from train set for validatoin
tes_start_idx = val_stop_idx - encoder_len
tes_stop_idx = total_time_idx-1

print(tr_stop_idx)
print(val_start_idx)
print(val_stop_idx)
print(tes_start_idx)
print(tes_stop_idx)

17353
17345
22303
22295
24789


In [ ]:
train_time_idx = [*range(first_idx, first_idx+tr_stop_idx+1, 1)]
print(train_time_idx[0], train_time_idx[-1])
val_time_idx = [*range(first_idx+val_start_idx, first_idx+val_stop_idx+1, 1)]
print(val_time_idx[0], val_time_idx[-1])
test_time_idx = [*range(first_idx+tes_start_idx, first_idx+tes_stop_idx+1, 1)]
print(test_time_idx[0], test_time_idx[-1])

23 17376
17368 22326
22318 24812


# Cluster Aggregation

In [ ]:
cluster_aggregation

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0","inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_1",inflow_diff2_per1__kurtosis,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_3,inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__skewness,inflow_diff2_per1__count_above_mean
0,167,2021-01-07 23:00:00,50,59.0,107.0,38.0,12.0,117.0,122.0,41.0,...,-3.0,-4.786351,0.561537,-0.128245,-0.269178,-0.269166,-0.110965,-0.026810,-0.468564,0.0
1,168,2021-01-08 00:00:00,50,32.0,67.0,14.0,30.0,158.0,59.0,23.0,...,4.0,-10.000000,0.561537,-0.128245,-1.000000,-0.269166,-0.110965,-0.026810,-0.468564,1.0
2,169,2021-01-08 01:00:00,50,32.0,34.0,17.0,84.0,120.0,32.0,23.0,...,27.0,-18.000000,0.561537,-1.465116,-0.017442,-0.269166,-0.110965,-0.026810,0.670284,1.0
3,170,2021-01-08 02:00:00,50,24.0,24.0,4.0,162.0,63.0,32.0,8.0,...,25.0,-26.000000,0.891341,-0.741231,-0.406555,-0.406555,-0.110965,0.702128,1.279001,2.0
4,171,2021-01-08 03:00:00,50,9.0,12.0,12.0,243.0,24.0,24.0,27.0,...,31.0,-15.972136,1.663525,-0.630481,-0.373850,-0.373850,-0.110965,0.432843,1.254480,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247120,24808,2023-10-31 16:00:00,246,937.0,833.0,866.0,701.0,161.0,1060.0,1083.0,...,18.0,-155.248275,0.388287,-0.356099,0.135241,0.135241,0.053212,-0.073461,-0.347820,12.0
247121,24809,2023-10-31 17:00:00,246,884.0,797.0,842.0,560.0,75.0,937.0,979.0,...,-7.0,-110.884152,0.401069,-0.358260,0.141407,0.141407,0.077524,-0.060138,-0.317217,12.0
247122,24810,2023-10-31 18:00:00,246,843.0,766.0,772.0,324.0,37.0,884.0,959.0,...,68.0,38.090968,0.503437,-0.384094,0.142216,0.142216,0.048863,-0.096132,-0.416259,13.0
247123,24811,2023-10-31 19:00:00,246,735.0,764.0,701.0,161.0,32.0,843.0,791.0,...,-20.0,24.437373,0.209153,-0.370867,0.126136,0.126136,0.053884,-0.069905,-0.314286,13.0


In [ ]:
for c in cluster_aggregation:
    print(c)

time_idx
datetime
group
target
inflow_lag_0
inflow_lag_1
inflow_lag_2
inflow_lag_3
outflow_lag_0
outflow_lag_1
outflow_lag_2
outflow_lag_3
inflow
inflow_diff1_per1
inflow_diff1_per2
inflow_diff2_per1
inflow_lag_0__fft_coefficient__attr_"real"__coeff_0
inflow_lag_0__fft_coefficient__attr_"real"__coeff_1
inflow_lag_0__skewness
inflow_lag_0__count_above_mean
inflow_lag_0__autocorrelation__lag_1
inflow_lag_0__count_below_mean
inflow_lag_0__autocorrelation__lag_2
inflow_lag_0__autocorrelation__lag_3
inflow_lag_0__partial_autocorrelation__lag_2
inflow_lag_0__partial_autocorrelation__lag_1
inflow_lag_0__partial_autocorrelation__lag_3
inflow_lag_0__kurtosis
inflow_lag_1__fft_coefficient__attr_"real"__coeff_0
inflow_lag_1__fft_coefficient__attr_"real"__coeff_1
inflow_lag_1__skewness
inflow_lag_1__count_above_mean
inflow_lag_1__autocorrelation__lag_1
inflow_lag_1__count_below_mean
inflow_lag_1__autocorrelation__lag_2
inflow_lag_1__autocorrelation__lag_3
inflow_lag_1__partial_autocorrelation__lag

In [ ]:
print(cluster_aggregation['datetime'][:10].tolist())

[Timestamp('2021-01-07 23:00:00'), Timestamp('2021-01-08 00:00:00'), Timestamp('2021-01-08 01:00:00'), Timestamp('2021-01-08 02:00:00'), Timestamp('2021-01-08 03:00:00'), Timestamp('2021-01-08 04:00:00'), Timestamp('2021-01-08 05:00:00'), Timestamp('2021-01-08 06:00:00'), Timestamp('2021-01-08 07:00:00'), Timestamp('2021-01-08 08:00:00')]


In [ ]:
unique_values_group_cluster = list(cluster_aggregation['group'].unique())
print(len(unique_values_group_cluster), "\n")
print(unique_values_group_cluster)

10 

[50, 74, 120, 125, 141, 151, 232, 238, 239, 246]


In [ ]:
zones_time_idx_cluster = {}
for i in unique_values_group_cluster:
    df = cluster_aggregation[cluster_aggregation['group'] == i]
    time_idx_list = df['time_idx'].tolist()
    first_idx = time_idx_list[0]
    last_idx = time_idx_list[-1]
    zones_time_idx_cluster[i] = [first_idx, last_idx]
    print("Group ", i, " has ", df.shape[0], " number of rows in df_ts_fresh \n")
    print("First idx: ", first_idx, "; Last idx: ", last_idx, "\n")

Group  50  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  74  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  120  has  24715  number of rows in df_ts_fresh 

First idx:  98 ; Last idx:  24812 

Group  125  has  24670  number of rows in df_ts_fresh 

First idx:  143 ; Last idx:  24812 

Group  141  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  151  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  232  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  238  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  239  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  246  has  24788  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  24812 



In [ ]:
encoder_len = 8
pred_len = 1

total_time_idx = cluster_aggregation[cluster_aggregation.group == 141].shape[0]

first_idx = cluster_aggregation.loc[cluster_aggregation['group'] == 141, 'time_idx'].iloc[0]

tr_stop_idx = int(0.7*total_time_idx)
val_start_idx = tr_stop_idx - encoder_len
val_stop_idx = val_start_idx + int(0.2*total_time_idx) # use (tr_stop_idx - encoder_len) as encoder from train set for validatoin
tes_start_idx = val_stop_idx - encoder_len
tes_stop_idx = total_time_idx-1

print(tr_stop_idx)
print(val_start_idx)
print(val_stop_idx)
print(tes_start_idx)
print(tes_stop_idx)

17353
17345
22303
22295
24789


In [ ]:
train_time_idx = [*range(first_idx, first_idx+tr_stop_idx+1, 1)]
print(train_time_idx[0], train_time_idx[-1])
val_time_idx = [*range(first_idx+val_start_idx, first_idx+val_stop_idx+1, 1)]
print(val_time_idx[0], val_time_idx[-1])
test_time_idx = [*range(first_idx+tes_start_idx, first_idx+tes_stop_idx+1, 1)]
print(test_time_idx[0], test_time_idx[-1])

23 17376
17368 22326
22318 24812


# POI Aggregation

In [ ]:
poi_aggregation

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,inflow_lag_4,inflow_lag_5,...,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_1",inflow_diff2_per1__count_above_mean,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__skewness,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__kurtosis,inflow_diff2_per1__partial_autocorrelation__lag_2
0,169,2021-01-08 01:00:00,school,66.0,101.0,87.0,422.0,448.0,305.0,90.0,...,-60.456708,0.0,0.0,-0.081640,-0.027738,-0.081622,-0.496245,-0.263488,0.490017,-0.303142
1,170,2021-01-08 02:00:00,school,32.0,50.0,53.0,410.0,327.0,652.0,37.0,...,-14.000000,1.0,1.0,-1.000000,-0.027738,-0.081622,-0.496245,-0.263488,0.490017,-0.303142
2,171,2021-01-08 03:00:00,school,19.0,30.0,33.0,285.0,177.0,1094.0,24.0,...,-2.500000,1.0,2.0,-0.849256,-0.027738,-0.081622,0.633716,0.198511,0.490017,-0.303142
3,172,2021-01-08 04:00:00,school,25.0,19.0,25.0,143.0,101.0,1120.0,20.0,...,9.000000,2.0,2.0,-0.929158,-0.711447,-0.929158,-0.071375,0.749460,-2.711060,-0.303142
4,173,2021-01-08 05:00:00,school,105.0,30.0,32.0,69.0,50.0,1135.0,18.0,...,1.225425,3.0,2.0,-0.268487,-0.207884,-0.268487,-0.544485,-0.049543,-1.927011,-0.303142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246435,24808,2023-10-31 16:00:00,religious,732.0,998.0,848.0,906.0,1070.0,678.0,1169.0,...,-2348.787241,13.0,12.0,0.160769,-0.059342,0.160769,-0.004889,-0.165414,-1.039837,-0.196335
246436,24809,2023-10-31 17:00:00,religious,637.0,840.0,798.0,910.0,882.0,476.0,942.0,...,-1053.705532,14.0,11.0,0.129439,-0.185665,0.129439,-0.071656,-0.180286,-0.891656,-0.200398
246437,24810,2023-10-31 18:00:00,religious,602.0,697.0,737.0,951.0,879.0,297.0,895.0,...,-159.359368,15.0,10.0,0.142760,-0.226929,0.142760,-0.186733,-0.239184,-0.900303,-0.264965
246438,24811,2023-10-31 19:00:00,religious,452.0,665.0,678.0,998.0,942.0,124.0,933.0,...,-141.605817,14.0,11.0,0.139179,-0.186107,0.139179,-0.059585,-0.234582,-1.027312,-0.258970


In [ ]:
for c in poi_aggregation:
    print(c)

time_idx
datetime
group
target
inflow_lag_0
inflow_lag_1
inflow_lag_2
inflow_lag_3
inflow_lag_4
inflow_lag_5
outflow_lag_0
outflow_lag_1
outflow_lag_2
outflow_lag_3
outflow_lag_4
outflow_lag_5
inflow
inflow_diff1_per1
inflow_diff1_per2
inflow_diff2_per1
inflow_lag_0__fft_coefficient__attr_"real"__coeff_1
inflow_lag_0__skewness
inflow_lag_0__kurtosis
inflow_lag_0__count_above_mean
inflow_lag_0__count_below_mean
inflow_lag_0__autocorrelation__lag_1
inflow_lag_0__autocorrelation__lag_2
inflow_lag_0__autocorrelation__lag_3
inflow_lag_0__partial_autocorrelation__lag_1
inflow_lag_0__partial_autocorrelation__lag_2
inflow_lag_0__partial_autocorrelation__lag_3
inflow_lag_0__fft_coefficient__attr_"real"__coeff_0
inflow_lag_1__fft_coefficient__attr_"real"__coeff_1
inflow_lag_1__skewness
inflow_lag_1__kurtosis
inflow_lag_1__count_above_mean
inflow_lag_1__count_below_mean
inflow_lag_1__autocorrelation__lag_1
inflow_lag_1__autocorrelation__lag_2
inflow_lag_1__autocorrelation__lag_3
inflow_lag_1__par

In [ ]:
unique_values_group_poi = list(poi_aggregation['group'].unique())
print(len(unique_values_group_poi), "\n")
print(unique_values_group_poi)

10 

['school', 'school_uni', 'school_sec', 'school_pri', 'food', 'food_restaurant', 'food_morning', 'food_bar', 'attractions', 'religious']


In [ ]:
zones_time_idx_poi = {}
for i in unique_values_group_poi:
    df = poi_aggregation[poi_aggregation['group'] == i]
    time_idx_list = df['time_idx'].tolist()
    first_idx = time_idx_list[0]
    last_idx = time_idx_list[-1]
    zones_time_idx_poi[i] = [first_idx, last_idx]
    print("Group ", i, " has ", df.shape[0], " number of rows in df_ts_fresh \n")
    print("First idx: ", first_idx, "; Last idx: ", last_idx, "\n")

Group  school  has  24644  number of rows in df_ts_fresh 

First idx:  169 ; Last idx:  24812 

Group  school_uni  has  24644  number of rows in df_ts_fresh 

First idx:  169 ; Last idx:  24812 

Group  school_sec  has  24644  number of rows in df_ts_fresh 

First idx:  169 ; Last idx:  24812 

Group  school_pri  has  24644  number of rows in df_ts_fresh 

First idx:  169 ; Last idx:  24812 

Group  food  has  24644  number of rows in df_ts_fresh 

First idx:  169 ; Last idx:  24812 

Group  food_restaurant  has  24644  number of rows in df_ts_fresh 

First idx:  169 ; Last idx:  24812 

Group  food_morning  has  24644  number of rows in df_ts_fresh 

First idx:  169 ; Last idx:  24812 

Group  food_bar  has  24644  number of rows in df_ts_fresh 

First idx:  169 ; Last idx:  24812 

Group  attractions  has  24644  number of rows in df_ts_fresh 

First idx:  169 ; Last idx:  24812 

Group  religious  has  24644  number of rows in df_ts_fresh 

First idx:  169 ; Last idx:  24812 



In [ ]:
encoder_len = 8
pred_len = 1

total_time_idx = poi_aggregation[poi_aggregation.group == 'school'].shape[0]

first_idx = poi_aggregation.loc[poi_aggregation['group'] == 'school', 'time_idx'].iloc[0]

tr_stop_idx = int(0.7*total_time_idx)
val_start_idx = tr_stop_idx - encoder_len
val_stop_idx = val_start_idx + int(0.2*total_time_idx) # use (tr_stop_idx - encoder_len) as encoder from train set for validatoin
tes_start_idx = val_stop_idx - encoder_len
tes_stop_idx = total_time_idx-1

print(tr_stop_idx)
print(val_start_idx)
print(val_stop_idx)
print(tes_start_idx)
print(tes_stop_idx)

17250
17242
22170
22162
24643


In [ ]:
train_time_idx = [*range(first_idx, first_idx+tr_stop_idx+1, 1)]
print(train_time_idx[0], train_time_idx[-1])
val_time_idx = [*range(first_idx+val_start_idx, first_idx+val_stop_idx+1, 1)]
print(val_time_idx[0], val_time_idx[-1])
test_time_idx = [*range(first_idx+tes_start_idx, first_idx+tes_stop_idx+1, 1)]
print(test_time_idx[0], test_time_idx[-1])

169 17419
17411 22339
22331 24812


In [ ]:
school_test = poi_aggregation[(poi_aggregation['group'] == 'school') & (poi_aggregation['time_idx'].isin(test_time_idx))]
school_test

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,inflow_lag_4,inflow_lag_5,...,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_1",inflow_diff2_per1__count_above_mean,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__skewness,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__kurtosis,inflow_diff2_per1__partial_autocorrelation__lag_2
22162,22331,2023-07-20 11:00:00,school,2105.0,2069.0,2084.0,2121.0,1812.0,2531.0,1881.0,...,-403.870671,11.0,14.0,-0.061801,-0.484652,-0.061801,0.370719,-0.150030,-0.053787,-0.154439
22163,22332,2023-07-20 12:00:00,school,2035.0,2132.0,2122.0,2336.0,1828.0,2365.0,1907.0,...,-286.563585,12.0,13.0,-0.031881,-0.399111,-0.031881,0.276982,-0.162086,0.033755,-0.163268
22164,22333,2023-07-20 13:00:00,school,2090.0,2202.0,2094.0,2353.0,1996.0,2408.0,1971.0,...,-500.412461,12.0,13.0,-0.024921,-0.354417,-0.024921,0.351286,-0.286698,0.311451,-0.287497
22165,22334,2023-07-20 14:00:00,school,2478.0,2155.0,2135.0,2288.0,2069.0,2216.0,2042.0,...,-245.992299,12.0,13.0,0.010101,-0.364790,0.010101,0.228006,-0.262154,-0.097396,-0.262282
22166,22335,2023-07-20 15:00:00,school,2542.0,2717.0,2577.0,2586.0,2132.0,1415.0,2419.0,...,-719.475289,13.0,12.0,0.188343,-0.231834,0.188343,-0.131049,-0.147982,-0.489267,-0.190202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24639,24808,2023-10-31 16:00:00,school,2251.0,2650.0,2666.0,3112.0,2535.0,399.0,3292.0,...,-778.818047,15.0,10.0,0.109609,0.122918,0.109609,-0.367134,-0.407201,0.049412,-0.424313
24640,24809,2023-10-31 17:00:00,school,1939.0,2331.0,2470.0,2708.0,2586.0,195.0,2870.0,...,-460.380523,16.0,9.0,0.007155,0.146568,0.007155,-0.511872,-0.404804,0.423025,-0.404876
24641,24810,2023-10-31 18:00:00,school,1722.0,2082.0,2192.0,2637.0,2717.0,84.0,2763.0,...,105.286661,15.0,10.0,0.064990,0.010944,0.064990,-0.642079,-0.575762,0.919060,-0.582446
24642,24811,2023-10-31 19:00:00,school,1406.0,1957.0,1981.0,2652.0,2650.0,61.0,2819.0,...,43.338913,14.0,11.0,0.030318,0.097774,0.030318,-0.493907,-0.540866,0.740993,-0.542283
